### Init ray

In [1]:
import ray
ray.init(address="auto", ignore_reinit_error=True)

2020-12-04 15:03:09,985	INFO worker.py:634 -- Connecting to existing Ray cluster at address: 192.168.8.51:6379


{'node_ip_address': '192.168.8.51',
 'raylet_ip_address': '192.168.8.51',
 'redis_address': '192.168.8.51:6379',
 'object_store_address': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044/sockets/raylet',
 'webui_url': '192.168.8.51:8265',
 'session_dir': '/tmp/ray/session_2020-12-04_15-00-53_639169_14044',
 'metrics_export_port': 65175}

### Load default config

In [2]:
# load defaulft config
import yaml
import os
import time
import pandas as pd
import random

config_path = './configs/default_cifar.yml'

with open(config_path) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)

# create base dir and gr
if os.path.exists(config["PROJECT"]["project_dir"]) is False:
    os.mkdir(config["PROJECT"]["project_dir"])

if os.path.exists(config["PROJECT"]["group_dir"]) is False:
    os.mkdir(config["PROJECT"]["group_dir"])

### Get the data to annotate

For DataNet we should pass the list of the annotated images

In [3]:
# Get the data to annotate

#############################################################################################
# LOAD DATA
#############################################################################################
from data_utils import CIFAR10Data
# Load data
cifar10_data = CIFAR10Data()
num_classes = len(cifar10_data.classes)
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reminiz/ReminizML2/python_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reminiz/ReminizML2/python_env/lib/pyt

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


In [4]:
indices = list(range(len(x_train)))
random.shuffle(indices)
labeled_set = indices
unlabeled_set = []

In [5]:
# test with all the images
NUM_IMAGES_TEST = len(x_test)
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
test_set = list(range(NUM_IMAGES_TEST))

In [6]:
config["NETWORK"]["INPUT_SIZE"] =  x_train[0].shape[0]
config["NETWORK"]["CLASSES"] = cifar10_data.classes

In [7]:
config

{'ACTIVE_ALGO': {'LOSSLEARNING': 1.0},
 'DATASET': {'height_shift_range': 4,
  'horizontal_flip': True,
  'width_shift_range': 4},
 'NETWORK': {'CLASSES': ['plane',
   'car',
   'bird',
   'cat',
   'deer',
   'dog',
   'frog',
   'horse',
   'ship',
   'truck'],
  'INPUT_SIZE': 32,
  'MARGIN': 1.0,
  'embedding_size': 128},
 'PROJECT': {'Backbone': 'resnet18',
  'dataset_name': 'CIFAR',
  'group': 'Classif_all_data',
  'group_dir': '/mnt/Ressources/Andres/Temp_active/runs/Classif_all_data',
  'project': 'Active_Learning_CIFAR',
  'project_dir': '/mnt/Ressources/Andres/Temp_active/runs',
  'source': 'CIFAR'},
 'RUNS': {'ADDENDUM': 1000,
  'CYCLES': 1,
  'SUBSET': -1,
  'TRIALS': 1,
  'test_each': 1},
 'TEST': {'batch_size': 128},
 'TRAIN': {'Data_augementation': True,
  'EPOCH_SLIT': 10,
  'EPOCH_WARMUP': 2,
  'EPOCH_WHOLE': 40,
  'MILESTONES': [25, 35],
  'batch_size': 128,
  'gamma': 0.1,
  'lr': 0.001,
  'start_epoch': 0,
  'transfer_weight_path': False,
  'w_c_loss': 1.0,
  'w_l_lo

### Set Configuration from the data

In [8]:
from train_agent_cifar import Active_Learning_train
#from inference_agent_cifar import Active_Learning_inference

In [9]:
NetworkActor =  Active_Learning_train.remote(config, labeled_set, test_set,  0, False)
NetworkActor.start_training.remote()


In [10]:
NetworkActor.__ray_terminate__.remote()
    

ObjectRef(4663044589b22052cd777f160400000001000000)

In [11]:
NetworkActor

Actor(Active_Learning_train,cd777f1604000000)